<a href="https://colab.research.google.com/github/cosmina98/PhD/blob/main/Work_in_progress/MessPass_Gated_Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [2]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
!pip install plotnine --quiet
!pip install torch_geometric
! install torch_geometric
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.1+cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
install: missing destination file operand after 'torch_geometric'
Try 'install --help' for more information.


In [3]:
import torch_geometric
from torch_geometric.datasets import Planetoid
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

In [4]:
import torch_geometric 
from typing import Callable, List, Optional, Tuple
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch import Tensor
from torch.optim import Optimizer
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv,ChebConv
#from torch_geometric.utils import accuracy
from typing_extensions import Literal, TypedDict
from torch_geometric.utils import get_laplacian
from torch_geometric.utils import add_self_loops, degree
import numpy as np
import pandas as pd
from plotnine import ggplot, geom_line, aes, xlab, theme, element_blank, ggtitle
import scipy.sparse as sparse
from sklearn.model_selection import KFold
from torch_geometric.utils import to_scipy_sparse_matrix
from dataclasses import dataclass

In [10]:
# libary imports 
import os
import torch
from torch import nn
import torchvision
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.datasets as datasets

from sklearn import metrics
from sklearn.metrics import accuracy_score

from sklearn import decomposition
from sklearn import manifold
from tqdm.notebook import trange, tqdm

import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import matplotlib.pyplot as plt
import numpy as np
import copy
import random
import time
import pandas as pd

import itertools
from functools import reduce
import operator
import torch.utils.data as data_utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import multilabel_confusion_matrix
from torch.utils.data import Subset

from torch.optim.adadelta import Adadelta
from torch.optim.adagrad import Adagrad
from torch.optim.adam import Adam
from torch.optim.adamax import Adamax
from torch.optim.adamw import AdamW
from torch.optim.optimizer import Optimizer
from torch.optim.sgd import SGD

import torch_geometric 
from typing import Callable, List, Optional, Tuple
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch import Tensor
from torch.optim import Optimizer
from torch_geometric.data import Data
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv,ChebConv
from typing_extensions import Literal, TypedDict
from torch_geometric.utils import get_laplacian
import numpy as np
import pandas as pd
from plotnine import ggplot, geom_line, aes, xlab, theme, element_blank, ggtitle
import scipy.sparse as sparse
from sklearn.model_selection import KFold
from torch_geometric.utils import to_scipy_sparse_matrix
from dataclasses import dataclass
import torch_geometric.utils
#from torch_geometric.utils import accuracy


In [5]:
device = torch.device('cuda' if torch.cuda.is_available()  else 'cpu')


#Dataset imports

# Training and testing targets for each dataset

In [6]:
dataset=list(range(3))
i=0
for dataset_name in ['Cora','CiteSeer','PubMed']:
    transform = T.Compose([T.TargetIndegree(),
])
    path = osp.join('data', dataset_name)
    dataset[i] =Planetoid(path, dataset_name, transform=transform)
    num_nodes = dataset[i].data.num_nodes
    num_edges = dataset[i].data.num_edges // 2
    train_len = dataset[i][0].train_mask.sum()
    val_len = dataset[i][0].val_mask.sum()
    test_len = dataset[i][0].test_mask.sum()
    other_len = num_nodes - train_len - val_len - test_len
    print(f"Dataset: {dataset[i].name}")
    print(f"Num. nodes: {num_nodes} (train={train_len}, val={val_len}, test={test_len}, other={other_len})")
    print(f"Num. edges: {num_edges}")
    print(len(dataset[i][0].edge_attr))
    print(f"Num. node features: {dataset[i].num_node_features}")
    print(f"Num. classes: {dataset[i].num_classes}")
    print(f"Dataset len.: {dataset[i].len()}")
    i=i+1
    print(" ")


Dataset: Cora
Num. nodes: 2708 (train=140, val=500, test=1000, other=1068)
Num. edges: 5278
10556
Num. node features: 1433
Num. classes: 7
Dataset len.: 1
 
Dataset: CiteSeer
Num. nodes: 3327 (train=120, val=500, test=1000, other=1707)
Num. edges: 4552
9104
Num. node features: 3703
Num. classes: 6
Dataset len.: 1
 
Dataset: PubMed
Num. nodes: 19717 (train=60, val=500, test=1000, other=18157)
Num. edges: 44324
88648
Num. node features: 500
Num. classes: 3
Dataset len.: 1
 


In [7]:
d={0:{}, 1:{}, 2:{}}
for i in range(3):
    data=dataset[i]
    X_train=data[0].x[data[0].train_mask]
    d[i]['X_train']=X_train
    y_train=data[0].y[data[0].train_mask]
    d[i]['y_train']=y_train
    X_test=data[0].x[data[0].test_mask]
    d[i]['X_test']=X_test
    y_test=data[0].y[data[0].test_mask]
    d[i]['y_test']=y_test
    X_val=data[0].x[data[0].val_mask]
    d[i]['X_val']=X_val
    y_val=data[0].y[data[0].val_mask]
    d[i]['y_val']=X_val


#Message Passing 

In [8]:
from torch_scatter import scatter_mean,scatter_add
from torch_geometric.nn.inits import uniform
dataset[0].data
x=dataset[0].data.x
edge_index=dataset[0].data.edge_index
edge_weight = dataset[0][0].edge_attr
x_j = x[edge_index[0]]  # Source node features [num_edges, num_features]
x_i = x[edge_index[1]]  # Target node features [num_edges, num_features]

In [51]:
torch.manual_seed(0)


class GatedGraphConv(nn.Module):
    
    def __init__(self, in_channels, out_channels,num_layers=5,bias=True):
        super().__init__()  
        #initialisation
        self.lin = nn.Linear(in_channels, out_channels, bias=False)
        self.flow = 'source_to_target'

        self.out_channels=out_channels
        self.num_layers = num_layers
        self.W = nn.Parameter(Tensor(num_layers, out_channels, out_channels))
        self.rnn = torch.nn.GRUCell(out_channels, out_channels, bias=bias)
        self.reset_parameters()


    def sum_aggregate(self,x,edge_index):
        row,col=edge_index
        tmp = torch.index_select(x, 0, col) # shape [num_edges, embed_size)
        aggr2 = scatter_add(tmp,row,0)
        return aggr2

    def reset_parameters(self):
        uniform(self.out_channels, self.W)
        self.rnn.reset_parameters()

    def forward(self, x, edge_index,edge_weight):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        # Step 1: Check of embedding size is larger than the target output channels 
        #Otherwise error; Trying to create tensor with negative dimension 
        if x.size(-1) > self.out_channels:
            raise ValueError('The number of input channels is not allowed to '
                             'be larger than the number of output channels')
        #Fill x with 0 along the y axis to match the out_channels
        if x.size(-1) < self.out_channels:
            zero = x.new_zeros(x.size(0), self.out_channels - x.size(-1))
            x = torch.cat([x, zero], dim=1)
   
        
        for i in range(self.num_layers):
            #Multiply x by their weights 
            m = torch.matmul(x, self.W[i])
            # propagate_type: (x: Tensor, edge_weight: OptTensor)
           # m = self.propagate(edge_index, x=m, edge_weight=edge_weight,
                               #size=None)
            out=self.collect(m,edge_index)
            x_j=out['x_j']
            m = self.message(x_j,edge_weight)
            m= self.sum_aggregate(m,edge_index)
            x = self.rnn(m, x)

        return x

    def collect(self ,x,edge_index):
        i, j = (1, 0) if self.flow == 'source_to_target' else (0, 1)
        out={}
        # 2. construct message x_j, x_i. Both with shape [num_edge, embed_size]
        out['x_j'] = x.index_select(0, edge_index[i]) 
        out['x_i'] = x.index_select(0, edge_index[j])
        out['edge_index_i'] = edge_index[i] # Source node edges 
        out['edge_index_j'] = edge_index[j]   # Target node edges 
        return out  

   
    def message(self, x_j,edge_weight=None):
        # x_j has shape [E, out_channels]
        return  x_j if edge_weight is None else edge_weight.view(-1, 1) * x_j

    def __repr__(self):
        return '{}({}, num_layers={})'.format(self.__class__.__name__,
                                              self.out_channels,
                                              self.num_layers)
    
    #'not needed'
    def update(self, aggr_out):
    # aggr_out has shape [num_nodes, out_channels]

    # Step 5: Return new node embeddings.
        return aggr_out

In [55]:
mask = F.dropout(torch.ones(edge_index.shape[1]), 0.5) > 0
edge_index_to_use = edge_index[:, mask]
edge_attr_to_use = edge_weight[mask] if edge_weight is not None else None
(edge_index_to_use.shape)

torch.Size([2, 5217])

In [56]:
conv = GatedGraphConv(x.shape[1], 6000)
m=conv(x, edge_index_to_use,edge_attr_to_use)
m.shape

torch.Size([5217, 6000])
torch.Size([5217, 6000])
torch.Size([5217, 6000])
torch.Size([5217, 6000])
torch.Size([5217, 6000])


torch.Size([2708, 6000])

In [58]:
m

tensor([[-1.5037e-02, -9.6160e-03, -1.0868e-02,  ..., -6.6930e-05,
          3.7338e-03, -1.8375e-02],
        [-1.6976e-02, -1.4247e-02, -9.1350e-03,  ..., -3.3574e-03,
          7.9774e-03, -1.4439e-02],
        [-1.5933e-02, -1.6878e-02, -1.8099e-02,  ..., -1.5389e-02,
          1.1828e-02, -1.6820e-02],
        ...,
        [-1.6636e-02, -1.1975e-02, -1.3162e-02,  ..., -2.7292e-03,
          9.5077e-03, -1.4799e-02],
        [-1.7691e-02, -1.2838e-02, -1.3966e-02,  ..., -1.9159e-03,
          2.2900e-03, -1.7274e-02],
        [-1.5633e-02, -9.6784e-03, -9.8897e-03,  ..., -2.7824e-03,
          3.7336e-03, -1.7164e-02]], grad_fn=<AddBackward0>)

In [79]:
edge_weight.view(-1, 1)

tensor([[0.0179],
        [0.0238],
        [0.0179],
        ...,
        [0.1964],
        [0.0238],
        [0.0238]])

In [81]:
edge_weight.shape

torch.Size([10556, 1])

In [13]:
print(conv.__repr__())

GatedGraphConv(14333, num_layers=5)
